<a href="https://colab.research.google.com/github/Vishak05/SDC-GenAI/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# GAN with MNIST using Keras
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Load MNIST
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.0
X_train = X_train.reshape(-1, 784)

# Generator
def build_generator():
    model = Sequential()
    model.add(Dense(256, input_dim=100))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization())
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(Dense(784, activation='tanh'))
    return model

# Discriminator
def build_discriminator():
    model = Sequential()
    model.add(Dense(512, input_dim=784))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Models
optimizer = Adam(0.0002, 0.5)
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator()
z = np.random.normal(0, 1, (1, 100))
img = generator.predict(z)

discriminator.trainable = False
gan = Sequential([generator, discriminator])
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

# Training
epochs = 1000
batch_size = 128

for epoch in range(epochs):
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real = X_train[idx]
    fake = generator.predict(np.random.normal(0, 1, (batch_size, 100)))

    X_combined = np.concatenate([real, fake])
    y_combined = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

    d_loss = discriminator.train_on_batch(X_combined, y_combined)

    z = np.random.normal(0, 1, (batch_size, 100))
    y_gen = np.ones((batch_size, 1))
    g_loss = gan.train_on_batch(z, y_gen)

    if epoch % 200 == 0:
        print(f"Epoch {epoch} - D Loss: {d_loss[0]}, G Loss: {g_loss}")
